## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

In [2]:
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')
calib_images=[]
    
def calibrate_Camera_Images(images,chess_coordinate_x,chess_coordinate_y):
    # Step through the list and search for chessboard corners
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chess_coordinate_y*chess_coordinate_x,3), np.float32)
    objp[:,:2] = np.mgrid[0:chess_coordinate_x,0:chess_coordinate_y].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    obj_points = [] # 3d points in real world space
    img_points = [] # 2d points in image plane.
    calib_images_array=[]
    
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (chess_coordinate_x,chess_coordinate_y),None)

        # If found, add object points, image points
        if ret == True:
            obj_points.append(objp)
            img_points.append(corners)
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (chess_coordinate_x,chess_coordinate_y), corners, ret)
            #plt.figure()
            #plt.imshow(img)
            calib_images_array.append(img)
            #cv2.imshow('img',img)
            #cv2.waitKey(500)
    #cv2.destroyAllWindows()
    return obj_points,img_points,calib_images_array

chess_coordinate_x=9
chess_coordinate_y=6
objpoints,imgpoints,calib_images=calibrate_Camera_Images(images,chess_coordinate_x,chess_coordinate_y)

In [3]:
for img in calib_images:
    plt.figure()
    plt.subplot(221)
    plt.imshow(img)

# Apply a distortion correction to raw images.

In [ ]:
import pickle
#calculate distortion values
img = cv2.imread('../camera_cal/calibration1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
dst = cv2.undistort(img, mtx, dist, None, mtx)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "calibration.p", "wb" ) )

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.subplots_adjust(hspace = .2, wspace=.05)
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

# Use color transforms, gradients, etc., to create a thresholded binary image.

In [ ]:
img_test = cv2.imread('../test_images/test1.jpg')
def undistort(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def undist_color_transform_binary(img):
    img_test=undistort(img)
    hls_test=cv2.cvtColor(img_test,cv2.COLOR_BGR2HLS)
    s_channel=hls_test[:,:,2]
    thresh = (90, 255)
    binary = np.zeros_like(s_channel)
    binary[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary
binary=undist_color_transform_binary(img_test)
plt.imshow(binary,cmap='gray')

# Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
img_test = cv2.imread('../test_images/test1.jpg')
img_size = (img_test.shape[1], img_test.shape[0])
offset=100
src = np.float32([(610,410),(700,410),(1100,700),(220,700)])
#dst = np.float32([[offset, offset], [img_size[0]-offset, offset],[img_size[0]-offset, img_size[1]-offset],[offset, img_size[1]-offset]])

#M=cv2.getPerspectiveTransform(src,dst)
#warped = cv2.warpPerspective(undistor_image,M,img_size)

# Visualize unwarp
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img_test)
x = [src[0][0],src[2][0],src[3][0],src[1][0],src[0][0]]
y = [src[0][1],src[2][1],src[3][1],src[1][1],src[0][1]]
ax1.plot(x, y, color='#22bb44',linewidth=4, zorder=2)
ax1.set_title('test image', fontsize=30)

# Detect lane pixels and fit to find the lane boundary.

# Determine the curvature of the lane and vehicle position with respect to center.

# Warp the detected lane boundaries back onto the original image.

# Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.